In [85]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [86]:
science_scores_df = pd.read_csv('8th_Grade_Science_Proficiency.csv')
science_scores_df.head()

,Jurisdiction,MN,SigDiff,SigSymbol,AB,AP
0,Utah,166,13,▲,82,50
1,DoDEA,166,13,▲,84,46
2,New Hampshire,165,12,▲,81,46
3,Vermont,163,10,▲,79,44
4,Minnesota,162,9,▲,76,45


In [87]:
science_scores_df = science_scores_df.rename(columns={'Jurisdiction': 'State',
                                                     'MN': 'Score (0-300)',
                                                     'SigDiff': 'Significant Difference',
                                                     'AB': 'Above Basic (%)',
                                                     'AP': 'Above Proficient (%)'})
science_scores_df.head()

,State,Score (0-300),Significant Difference,SigSymbol,Above Basic (%),Above Proficient (%)
0,Utah,166,13,▲,82,50
1,DoDEA,166,13,▲,84,46
2,New Hampshire,165,12,▲,81,46
3,Vermont,163,10,▲,79,44
4,Minnesota,162,9,▲,76,45


In [88]:
science_scores_df = science_scores_df.drop(columns='SigSymbol')
science_scores_df.head()

,State,Score (0-300),Significant Difference,Above Basic (%),Above Proficient (%)
0,Utah,166,13,82,50
1,DoDEA,166,13,84,46
2,New Hampshire,165,12,81,46
3,Vermont,163,10,79,44
4,Minnesota,162,9,76,45


In [89]:
science_scores_df = science_scores_df[~science_scores_df['State'].str.contains('District of Columbia')]
science_scores_df = science_scores_df[~science_scores_df['State'].str.contains('Puerto Rico')]
science_scores_df = science_scores_df[~science_scores_df['State'].str.contains('DoDEA')]
science_scores_df = science_scores_df[~science_scores_df['State'].str.contains('Pennsylvania')]
science_scores_df = science_scores_df[~science_scores_df['State'].str.contains('Louisiana')]
science_scores_df = science_scores_df[~science_scores_df['State'].str.contains('Colorado')]
science_scores_df = science_scores_df[~science_scores_df['State'].str.contains('Alaska')]
science_scores_df.head()

,State,Score (0-300),Significant Difference,Above Basic (%),Above Proficient (%)
0,Utah,166,13,82,50
2,New Hampshire,165,12,81,46
3,Vermont,163,10,79,44
4,Minnesota,162,9,76,45
5,Massachusetts,162,9,75,44


In [90]:
with open('states.json') as f:
    geojson = json.load(f)

In [91]:
state_lines_df = json_normalize(geojson["features"])
state_lines_df.head()

,geometry.coordinates,geometry.type,id,properties.density,properties.name,type
0,"[[[-87.359296, 35.00118], [-85.606675, 34.9847...",Polygon,01,94.65,Alabama,Feature
1,"[[[-109.042503, 37.000263], [-109.04798, 31.33...",Polygon,04,57.05,Arizona,Feature
2,"[[[-94.473842, 36.501861], [-90.152536, 36.496...",Polygon,05,56.43,Arkansas,Feature
3,"[[[-123.233256, 42.006186], [-122.378853, 42.0...",Polygon,06,241.70,California,Feature
4,"[[[-107.919731, 41.003906], [-105.728954, 40.9...",Polygon,08,49.33,Colorado,Feature


In [92]:
state_lines_df = state_lines_df[['geometry.type', 'geometry.coordinates', 'properties.name']]

In [93]:
state_lines_df = state_lines_df.rename(columns={'properties.name': 'State',
                                                'geometry.type': 'Type',
                                                'geometry.coordinates': 'Coordinates'})
state_lines_df.head()

,Type,Coordinates,State
0,Polygon,"[[[-87.359296, 35.00118], [-85.606675, 34.9847...",Alabama
1,Polygon,"[[[-109.042503, 37.000263], [-109.04798, 31.33...",Arizona
2,Polygon,"[[[-94.473842, 36.501861], [-90.152536, 36.496...",Arkansas
3,Polygon,"[[[-123.233256, 42.006186], [-122.378853, 42.0...",California
4,Polygon,"[[[-107.919731, 41.003906], [-105.728954, 40.9...",Colorado


In [94]:
learning_by_state_df = state_lines_df.merge(science_scores_df,how='left', on='State')
learning_by_state_df

,Type,Coordinates,State,Score (0-300),Significant Difference,Above Basic (%),Above Proficient (%)
0,Polygon,"[[[-87.359296, 35.00118], [-85.606675, 34.9847...",Alabama,141,-12,54,21
1,Polygon,"[[[-109.042503, 37.000263], [-109.04798, 31.33...",Arizona,148,-5,61,25
2,Polygon,"[[[-94.473842, 36.501861], [-90.152536, 36.496...",Arkansas,148,-4,62,28
3,Polygon,"[[[-123.233256, 42.006186], [-122.378853, 42.0...",California,143,-10,56,24
4,Polygon,"[[[-107.919731, 41.003906], [-105.728954, 40.9...",Colorado,NaN,NaN,NaN,NaN
5,Polygon,"[[[-73.053528, 42.039048], [-71.799309, 42.022...",Connecticut,155,2,70,35
6,Polygon,"[[[-75.414089, 39.804456], [-75.507197, 39.683...",Delaware,149,-4,62,29
7,Polygon,"[[[-77.035264, 38.993869], [-76.909294, 38.895...",District of Columbia,NaN,NaN,NaN,NaN
8,Polygon,"[[[-85.497137, 30.997536], [-85.004212, 31.003...",Florida,153,#,66,33
9,Polygon,"[[[-83.109191, 35.00118], [-83.322791, 34.7875...",Georgia,152,-1,65,31


In [98]:
# filling NaN with blank strings to work with graph
learning_by_state_df = learning_by_state_df.fillna('')

# Filling # in Florida with 0 because # means not significantly different than US average
learning_by_state_df = learning_by_state_df.replace('#', 0)
learning_by_state_df

,Type,Coordinates,State,Score (0-300),Significant Difference,Above Basic (%),Above Proficient (%)
0,Polygon,"[[[-87.359296, 35.00118], [-85.606675, 34.9847...",Alabama,141,-12,54,21
1,Polygon,"[[[-109.042503, 37.000263], [-109.04798, 31.33...",Arizona,148,-5,61,25
2,Polygon,"[[[-94.473842, 36.501861], [-90.152536, 36.496...",Arkansas,148,-4,62,28
3,Polygon,"[[[-123.233256, 42.006186], [-122.378853, 42.0...",California,143,-10,56,24
4,Polygon,"[[[-107.919731, 41.003906], [-105.728954, 40.9...",Colorado,,,,
5,Polygon,"[[[-73.053528, 42.039048], [-71.799309, 42.022...",Connecticut,155,2,70,35
6,Polygon,"[[[-75.414089, 39.804456], [-75.507197, 39.683...",Delaware,149,-4,62,29
7,Polygon,"[[[-77.035264, 38.993869], [-76.909294, 38.895...",District of Columbia,,,,
8,Polygon,"[[[-85.497137, 30.997536], [-85.004212, 31.003...",Florida,153,0,66,33
9,Polygon,"[[[-83.109191, 35.00118], [-83.322791, 34.7875...",Georgia,152,-1,65,31


In [99]:
# Write function to create geoJSON dictionary from dataframe
def df_to_geojson(df, properties):
    
    # Create Python dictionary to hold geoJSON information
    geojson = {'type':'FeatureCollection', 'features':[]}
    
    # Loop through the rows of the dataframe
    for _, row in df.iterrows():
        
        # Create feature for each object
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':[],
                               'coordinates':[]}}
        
        # Place geometry coordinates into geometry
        feature['geometry']['type'] = row['Type']
        feature['geometry']['coordinates'] = row['Coordinates']
        
        # Go through each item in columns list and append that to the properties section
        for prop in properties:
            feature['properties'][prop] = row[prop]
            
        # Append feature to dictionary
        geojson['features'].append(feature)
        
    # Return geoJSON dictionary
    return geojson

In [100]:
# Define list of properties
col = ["State", "Score (0-300)", "Significant Difference", "Above Basic (%)", "Above Proficient (%)"]

# Use function to Create geoJSON file
new_geoJSON = df_to_geojson(learning_by_state_df, col)

# Print geoJSON file
new_geoJSON

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'State': 'Alabama',
    'Score (0-300)': '141',
    'Significant Difference': '-12',
    'Above Basic (%)': '54',
    'Above Proficient (%)': '21'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-87.359296, 35.00118],
      [-85.606675, 34.984749],
      [-85.431413, 34.124869],
      [-85.184951, 32.859696],
      [-85.069935, 32.580372],
      [-84.960397, 32.421541],
      [-85.004212, 32.322956],
      [-84.889196, 32.262709],
      [-85.058981, 32.13674],
      [-85.053504, 32.01077],
      [-85.141136, 31.840985],
      [-85.042551, 31.539753],
      [-85.113751, 31.27686],
      [-85.004212, 31.003013],
      [-85.497137, 30.997536],
      [-87.600282, 30.997536],
      [-87.633143, 30.86609],
      [-87.408589, 30.674397],
      [-87.446927, 30.510088],
      [-87.37025, 30.427934],
      [-87.518128, 30.280057],
      [-87.655051, 30.247195],
      [-87.90699, 30.411504],
      [-87.93

In [101]:
# Create new file named scicenter.json
with open('complete.json', 'w') as fp:
    
    # format as JSON file from dictionary and indent as needed
    json.dump(new_geoJSON, fp)